# 事前準備：共通コードの実行
* このノートブックに接続したら，まずは以下の2つの共通コード（コードAとコードB）を実行する
* これらの共通コードを実行しないと，それ以降のコードが実行できないので注意する
* また，コードAとコードBは，ノートブックに接続するたび毎回実行すること（ノートブックに接続中は，何度も実行する必要はない）
* 共通コードの詳細についての説明は割愛する（簡単な説明は第2回の「[サンプルノートブック02](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook02.ipynb)」を参照）

In [ ]:
# コードA：日本語化ライブラリ導入
! pip install japanize-matplotlib | tail -n 1

In [25]:
# コードB：共通事前処理

# B1:余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib # matplotlib日本語化対応
import seaborn as sns

# B2:データフレーム表示用関数
from IPython.display import display

# B3:表示オプション調整
np.set_printoptions(suppress = True, precision = 3) #numpyの浮動小数点の表示精度
pd.options.display.float_format = '{:.3f}'.format #pandasでの浮動小数点の表示精度
pd.set_option('display.max_columns', None) #データフレームですべての列データを表示

# B4:グラフのデフォルトフォント指定
plt.rcParams['font.size'] = 14

# 乱数の種
random_seed = 123

# 分類木モデルの学習の実践 その1

## 用いるデータと分析の目的
* 例として，以下のURLにあるcsvファイルを用いる
>* https://bit.ly/43sIRq2
* このデータは，過去に沈没した客船の乗客に関する情報がまとめられたデータ（客船データと呼ぶ）である
>* 出典: 須藤秋良, 株式会社フレアリンク: スッキリわかるPythonによる機械学習入門, インプレス, 2020

|**列名**| **意味** |
|:--|:--|
|乗客ID| 乗客のID |
|生存| 0: 死亡 / 1: 生存 |
|チケットクラス| チケットの等級（1: 1等級 / 2: 2等級 / 3: 3等級） |
|性別| 乗客の性別（female: 女性 / male: 男性） |
|年齢| 乗客の年齢 |
|同乗1| 同乗した兄弟と配偶者の数 |
|同乗2| 同乗した親と子供の数 |
|チケットID| チケットのID |
|運賃| 乗船運賃 |
|部屋番号| 乗客の部屋番号 |
|港コード| 乗船した港を表すアルファベット |

* 客船データを用いて，「生存」を予測する分類木モデルを作成することを考える
  
**［以下のコードの処理内容］**
*  1行目: ファイルのURLを変数`url`に代入
*  2行目: pandasの`read_csv`関数を使って，ファイルをDataFrameとして読み込んで，変数`ship`に代入
*  3行目: `display`関数を使ってデータ（`ship`の内容）を表示


In [26]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
display(ship)

,乗客ID,生存,チケットクラス,性別,年齢,同乗1,同乗2,チケットID,運賃,部屋番号,港コード
0,1,0,3,male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,male,35.000,0,0,373450,8.050,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.000,0,0,211536,13.000,NaN,S
887,888,1,1,female,19.000,0,0,112053,30.000,B42,S
888,889,0,3,female,NaN,1,2,W./C. 6607,23.450,NaN,S
889,890,1,1,male,26.000,0,0,111369,30.000,C148,C


## 生存列のデータの種類の確認
* 今回は列「生存」のデータを目的変数（正解データ）として，分類木モデルの学習を行う
* そのために，まず生存列のデータがどのような値をとるのかを確認する
  
**［以下のコードの処理内容］**
*  1行目: ファイルのURLを変数`url`に代入
*  2行目: pandasの`read_csv`関数を使って，ファイルをDataFrameとして読み込んで，変数`ship`に代入
*  3行目: 列「生存」の値の頻度を抽出し，その結果を`display`関数で表示
>* `ship['生存'].value_counts()`で，DataFrameである`ship`の列「生存」にどんな値がそれぞれ何個あるのかを調べる
>* `value_counts`メソッドの戻り値がSeriesなので，pandasの`DataFrame`関数でDataFrameに変換 ⇒ `pd.DataFrame(ship['種類'].value_counts())`
>* 変換したDataFrameを`display`関数で表示

In [27]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
display(pd.DataFrame(ship['生存'].value_counts()))

,count
生存,
0,549
1,342


## 不均衡データ
* 目的変数とする生存列のデータは「0（死亡）」が549個で，「1（生存）」が342個と偏りがある ⇒ 不均衡データと呼ぶ
* 不均衡データだとうまく学習できないことがある
* さらに極端な例として，生存者が全体の5%だとする
* 年齢が0以上という条件で分類しても95%は正解になる
* この場合，実質的に学習しなくてもうまく分類できていることになる
* 学習時に不均衡データに対する対応が必要（後述）

## 欠損値の確認
* DataFrameに対して`isnull`メソッドを適用することで，欠損値の場所が`True`となる
* さらに，`isnull`メソッドの戻り値となるDataFrameに対して，`sum`メソッドを適用することで，各列の`True`（欠損値）の数が確認できる
* `sum`メソッドは合計を計算するメソッドであるが，計算対象のデータ型がboolの場合は，`True`を1，`False`を0として計算する
  
**［以下のコードの処理内容］**
*  1～2行目: ファイルをDataFrameとして読み込んで，変数`ship`に代入
*  3行目: `isnull`メソッドと`sum`メソッドを組み合わせて，各列の欠損値の数を計算して，`display`関数で表示する

In [28]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
display(pd.DataFrame(ship.isnull().sum(), columns = ['欠損値の数']))

,欠損値の数
乗客ID,0
生存,0
チケットクラス,0
性別,0
年齢,177
同乗1,0
同乗2,0
チケットID,0
運賃,0
部屋番号,687


## 欠損値への対処
* 部屋番号列の欠損値が非常に多いので，特徴量として使用しないことにする
* 年齢列のデータも欠損値が多く存在するが，生存列の値に関係がありそうなので，特徴量として使用することとし，欠損値は算術平均で置き換えることで対処する
* また，港コード列は欠損値が2件なので，最頻値に置き換えて対処する
* 最頻値は`mode`メソッドで求めることができる
* なお，`mode`メソッドの戻り値は，引数をDataFrame全体または列名のリストとするとDataFrameになる
* 一方，単独の列名を引数に指定するとSeriesになる
  
**［以下のコードの処理内容］**
* 1～2行目: ファイルをDataFrameとして読み込んで，変数`ship`に代入
* 3行目: `mode`メソッドを使って，港コード列（`ship['港コード']`）の最頻値を求め（データ型は Series），`print`関数で表示
* 実行結果より，港コードの最頻値が「S」であることが確認できる


In [29]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
print(ship['港コード'].mode())

0    S
Name: 港コード, dtype: object


* 年齢列の欠損値を算術平均（`ship['年齢'].mean()`）に置き換える
* 港コード列の欠損値を最頻値（`ship['港コード'].mode()[0]`）に置き換える
>* `ship['港コード'].mode()`がSeriesなので，`ship['港コード'].mode()[0]`とインデックスで指定する
* 欠損値の置き換えは，`fillna`メソッドを用いる
>* 書式: `fillna(置き換える値)`
  
**［以下のコードの処理内容］**
* 1～2行目: ファイルをDataFrameとして読み込んで，変数`ship`に代入
* 3行目: `fillna`メソッドを使って，年齢列（`ship['年齢']`）の欠損値を，その算術平均`ship['年齢'].mean()`に置き換える
* 4行目: `fillna`メソッドを使って，港コード列（`ship['港コード']`）の欠損値を，その最頻値`ship['港コード'].mode()[0]`に置き換える
* 3行目: `isnull`メソッドと`sum`メソッドを組み合わせて，各列の欠損値の数を計算して，`display`関数で表示する

In [30]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
ship['年齢'] = ship['年齢'].fillna(ship['年齢'].mean())
ship['港コード'] = ship['港コード'].fillna(ship['港コード'].mode()[0])
display(pd.DataFrame(ship.isnull().sum(), columns = ['欠損値の数']))

,欠損値の数
乗客ID,0
生存,0
チケットクラス,0
性別,0
年齢,0
同乗1,0
同乗2,0
チケットID,0
運賃,0
部屋番号,687


## データを説明変数（特徴量データ）と目的変数（正解データ）に分割
* これまでと同様にして，DataFrame `ship` を説明変数`x`と目的変数`t`に分割する  
* 説明変数には「チケットクラス」「年齢」「同乗1」「同乗2」「運賃」を用いる
* さらに，`x`と`t`を訓練データ`(x_train, t_train)`とテストデータ`(x_test, t_test)`に分割する
* 詳細は，[第3回のサンプルノートブック](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook03.ipynb)を参照
  
**［以下のコードの処理内容］**
*  1～4行目: ファイルをDataFrameとして読み込んで，変数`ship`に代入し，欠損値への対処を行う
*  5行目: 説明変数の列名を要素とするリスト`['チケットクラス', '年齢', '同乗1', '同乗2', '運賃']`を変数`col`に代入
*  6行目: DataFrame `ship` から,`ship[col]`で，説明変数の列だけ取り出し変数`x`に代入
*  7行目: DataFrame `ship` から,`ship['生存']`で目的変数の列「生存」だけ取り出し変数`t`に代入
*  8行目: `model_selection`モジュールの`train_test_split`関数の読み込み
*  9行目: `train_test_split`関数を使って説明変数データ`x`と目的変数データ（正解データ）`t`を訓練データとテストデータにそれぞれ分割
>*  `test_size = 0.2`として，訓練データを8割，テストデータを2割に分割
>*  `random_state = random_seed`として，乱数を固定する ⇒ 結果が同じになる
*  10行目: `print`関数で区切り線「================ x_train ================」を表示
*  11行目: `display`関数で変数`x_train`（訓練データの説明変数データ）を表示
*  12行目: `print`関数で区切り線「================ t_train ================」を表示
*  13行目: `display`関数で変数`t_train`（訓練データの目的変数データ）を表示
*  14行目: `print`関数で区切り線「================ x_test ================」を表示
*  15行目: `display`関数で変数`x_test`（テストデータの説明変数データ）を表示
*  16行目: `print`関数で区切り線「================ t_test ================」を表示
*  17行目: `display`関数で変数`t_test`（テストデータの目的変数データ）を表示

In [31]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
ship['年齢'] = ship['年齢'].fillna(ship['年齢'].mean())
ship['港コード'] = ship['港コード'].fillna(ship['港コード'].mode()[0])
col = ['チケットクラス', '年齢', '同乗1', '同乗2', '運賃']
x = ship[col]
t = ship['生存']
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size = 0.2, random_state = random_seed)
print('================ x_train ================')
display(x_train)
print('================ t_train ================')
display(pd.DataFrame(t_train))
print('================ x_test ================')
display(x_test)
print('================ t_test ================')
display(pd.DataFrame(t_test))

================ x_train ================


,チケットクラス,年齢,同乗1,同乗2,運賃
329,1,16.000,0,1,57.979
749,3,31.000,0,0,7.750
203,3,45.500,0,0,7.225
421,3,21.000,0,0,7.733
97,1,23.000,0,1,63.358
...,...,...,...,...,...
98,2,34.000,0,1,23.000
322,2,30.000,0,0,12.350
382,3,32.000,0,0,7.925
365,3,30.000,0,0,7.250


================ t_train ================


,生存
329,1
749,0
203,0
421,0
97,1
...,...
98,1
322,1
382,0
365,0


================ x_test ================


,チケットクラス,年齢,同乗1,同乗2,運賃
172,3,1.000,1,1,11.133
524,3,29.699,0,0,7.229
452,1,30.000,0,0,27.750
170,1,61.000,0,0,33.500
620,3,27.000,1,0,14.454
...,...,...,...,...,...
388,3,29.699,0,0,7.729
338,3,45.000,0,0,8.050
827,2,1.000,0,2,37.004
773,3,29.699,0,0,7.225


================ t_test ================


,生存
172,1
524,0
452,0
170,0
620,0
...,...
388,0
338,1
827,1
773,0


## 分類木モデルの学習と評価（1）

### 分類木モデルの学習と不均衡データへの対応
* 目的変数とする生存列のデータは「0（死亡）」が549個で，「1（生存）」が342個と偏りがある
* このような偏りのあるデータのことを不均衡データと呼ぶ
* 一般的に，不均衡データを使ってモデルの学習を行うと，良いモデルにならないことがある
* 極端な例として， 生存者が全体の5%だとすると，年齢が0以上という条件で分類しても95%は正解になる
* この場合，実質的に学習しなくてもうまく分類できていることになるが，これでは生存に影響を与えている要因（特徴量）を明らかにすることができないため，（精度は高いが）良いモデルとは言えない
* 不均衡データを使って学習する場合には，データの偏りの影響を取り除くための対応が必要となる
* 具体的には，`sklearn`（scikit-learn）の`tree`モジュールにおける`DecisionTreeClassifier`クラスから分類木のモデルオブジェクトを生成する際に，不均衡データの対処をするための引数`class_weight = 'balanced'`を指定する
* 詳細は省略するが，これによって，比率の大きいデータの影響を小さくし，比率の小さいデータの影響を大きくすることができる
* そのため，不均衡データの影響を取り除くことができる

### 分類木モデルの学習と評価
* 第3回と同様にして，分類木モデルの学習と評価を行う
* 詳細は，[第3回のサンプルノートブック](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook03.ipynb)を参照
* また，今回は訓練データとテストデータの両方で精度を計算し，比較する
  
**［以下のコードの処理内容］**
*  1～9行目: ファイルの読み込み，欠損値の対処，データの分割
*  10行目: `sklearn` (scikit-learn) の`tree`モジュールを読み込む 
*  11行目: `tree.DecisionTreeClassifier`で，分類木モデルの学習を行うためのオブジェクトを`tree`モジュールの`DecisionTreeClassifier`クラスから生成し，変数`model_tree_ship`に代入
>* `class_weight = 'balanced'`で不均衡データの影響を取り除く
>* `max_depth = 5`で最大深さを5とする
>* `random_state = random_seed`で乱数の種を指定する
*  12行目: `fit`メソッドで分類木モデルの学習を実行
>* 学習には訓練データ`x_train`, `t_train`を使う
*  13行目: `model_tree_ship.score(X = x_train, y = t_train)`で，訓練データから精度を計算して変数`score_train`に代入
*  14行目: `model_tree_ship.score(X = x_test, y = t_test)`で，テストデータに対する精度を計算して変数`score_test`に代入
*  15行目: `print`関数とf-stringを使って，訓練データに対する精度（`score_train`）とテストデータに対する精度（`score_test`）を表示
>* 「`:.3f`」で，小数点以下3桁まで表示


In [32]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
ship['年齢'] = ship['年齢'].fillna(ship['年齢'].mean())
ship['港コード'] = ship['港コード'].fillna(ship['港コード'].mode()[0])
col = ['チケットクラス', '年齢', '同乗1', '同乗2', '運賃']
x = ship[col]
t = ship['生存']
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size = 0.2, random_state = random_seed)
from sklearn import tree
model_tree_ship = tree.DecisionTreeClassifier(class_weight = 'balanced', max_depth = 5, random_state = random_seed)
model_tree_ship.fit(x_train, t_train)
score_train = model_tree_ship.score(x_train, t_train)
score_test = model_tree_ship.score(x_test, t_test)
print(f'訓練データの精度={score_train:.3f} / テストデータの精度={score_test:.3f}')

訓練データの精度=0.716 / テストデータの精度=0.665


### 最大深さによる精度の変化の確認と過学習
* これまでは，分類木モデルの学習の際には最大深さを適当に固定していた
* 分類木モデルの性能は最大深さが大きく影響するので，十分に検討した上で決める必要がある
* 以下は，分類木モデルの最大深さを1から14まで変化させたときの，訓練データとテストデータのそれぞれで計算した精度の推移の表とグラフである
  
|**深さ**| **訓練データの精度** | **テストデータの精度** |
|:--|:--|:--|
|1|0.618|0.598|
|2|0.653|0.642|
|3|0.654|0.648|
|4|0.636|0.581|
|5|0.716|0.665|
|6|0.721|0.665|
|7|0.767|0.682|
|8|0.774|0.659|
|9|0.810|0.704|
|10|0.824|0.642|
|11|0.843|0.642|
|12|0.879|0.676|
|13|0.900|0.659|
|14|0.916|0.648|

<img src="./fig/accuracy_graph.jpg" width="500">  

* 分類木の最大深さを大きくすると，分岐条件を多く設定することができるので，学習データを細かく分類することができる
* 分類木モデルは最大深さを大きくとることで複雑なモデルが表現できるので，表現力が豊かなモデルになる
* このことは訓練データの正解率の推移から確認できる（最大深さを大きくすると正解率も大きくなっている）
* 一方，テストデータの正解率は，最大深さが深くなってくると減少傾向になる
* この現象は，モデルを複雑にする（最大深さを大きくする）ことで，訓練データの詳細な特徴まで学習してしまい，訓練データに対して過度に当てはまったモデルになってしまったことが原因で生じている
* この現象のことを一般に**過学習**と呼ぶ
* 過学習によって，未知のデータ（テストデータ）に対して当てはまりが悪くなるので，テストデータの正解率は小さくなってしまう
* 以上より，モデルを必要以上に複雑にすると未知のデータに対するモデルの性能が低下するので，適切なモデルの複雑さにしてモデルを学習する必要がある
* また，過学習は決定木モデルだけでなく，すべての教師あり学習のモデルで起こる現象である
* 例えば，線形重回帰モデルでは説明変数（特徴量）の種類を増やすとモデルが複雑になり（表現力が豊かになり）過学習を起こしやすくなる


## 分類木モデルの学習と評価（2）

### モデルの性能向上
* 一般に，過学習を回避しつつ，良いモデル（未知データに対して当てはまりがよいモデル）を作るためには，様々なアプローチがあるが，ここでは，まず欠損値の対処方法の改善，及び使用する特徴量（説明変数）の再検討を試みる




### 年齢の分布に関するクロス集計
* 先ほどは年齢列の欠損値を算術平均に置き換えて対処していたが，この対処を改善することにする
* ここでは，年齢の分布がチケットクラスや生存者/死亡者（生存）の違いによって異なると考え，これら特徴量でグループに分け，それぞれのグループでの平均年齢（年齢の算術平均）を確認する
* この分析を一般には「クロス集計」と呼ぶ
* DataFrameに対するクロス集計は，`pivot_table`関数を用いる
* 詳細は「[ノートブック：SeriesとDataFrameの基本操作](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook_se_df.ipynb)」を参照
* ピボットテーブル集計の書式：`pd.pivot_table(df, index = 集計軸の列名1, columns = 集計軸の列名2, values = 集計対象の列, aggfunc = 関数（のリスト）, margins = [bool値])`
>* `aggfunc`のデフォルトは`mean`
>* `margins`を`True`にすることで列及び行ごとの合計も計算できる（デフォルトは`False`）
* 以下のように引数を指定することでチケットクラスと生存の値の組み合わせによる年齢の算術平均が確認できる
>*   `df`: `ship`
>*   `index = '集計軸の列名1'`: `index = '生存'`
>*   `column = '集計軸の列名2'`: `columns = 'チケットクラス'`）
>*   `values = 集計対象の列`: `values='年齢'`
>*   `aggfunc`と`margins`はデフォルト（指定しない）
  
**［以下のコードの処理内容］**
*  1～2行目: ファイルの読み込み
*  3行目: `pivot_table`関数を使って，チケットクラスと生存の値の組み合わせによる年齢の算術平均を集計し，`display`関数で表示

In [33]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
display(pd.pivot_table(ship, index = '生存', columns = 'チケットクラス', values = '年齢'))

チケットクラス,1,2,3
生存,,,
0,43.695,33.544,26.556
1,35.368,25.902,20.646


### 欠損値への対処の改善
* チケットクラスと生存/死亡の違いで平均年齢が違っている
* 年齢列の欠損値に対しては，該当する算術平均（小数点以下は切り捨てる）でそれぞれ置き換えることにする
* この置き換えには，DataFrameの`loc`メソッドを用いる
* 詳細は「[ノートブック：SeriesとDataFrameの基本操作](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook_se_df.ipynb)」を参照
* 具体的には，`df.loc[[条件式], '年齢'] = 置き換える値`とし，条件式を満たすデータの年齢の値を置き換える
* チケットクラス（1～3）と生存（0 or 1）の組み合わせの数の`[条件式]`を指定する
  
**［以下のコードの処理内容］**
*  1～2行目: ファイルの読み込み
*  3行目: チケットクラスが1，かつ生存が0，かつ年齢が欠損している（`NaN`の）データの年齢列の値を43にする
>* 条件式は`(ship['チケットクラス'] == 1) & (ship['生存'] == 0) & (ship['年齢'].isnull())`
*  4行目: チケットクラスが1，かつ生存が1，かつ年齢が欠損している（`NaN`の）データの年齢列の値を35にする
>* 条件式は`(ship['チケットクラス'] == 1) & (ship['生存'] == 1) & (ship['年齢'].isnull())`
*  5行目: チケットクラスが2，かつ生存が0，かつ年齢が欠損している（`NaN`の）データの年齢列の値を33にする
>* 条件式は`(ship['チケットクラス'] == 2) & (ship['生存'] == 0) & (ship['年齢'].isnull())`
*  6行目: チケットクラスが2，かつ生存が1，かつ年齢が欠損している（`NaN`の）データの年齢列の値を25にする
>* 条件式は`(ship['チケットクラス'] == 2) & (ship['生存'] == 1) & (ship['年齢'].isnull())`
*  7行目: チケットクラスが3，かつ生存が0，かつ年齢が欠損している（`NaN`の）データの年齢列の値を26にする
>* 条件式は`(ship['チケットクラス'] == 3) & (ship['生存'] == 0) & (ship['年齢'].isnull())`
*  8行目: チケットクラスが3，かつ生存が1，かつ年齢が欠損している（`NaN`の）データの年齢列の値を20にする
>* 条件式は`(ship['チケットクラス'] == 3) & (ship['生存'] == 1) & (ship['年齢'].isnull())`
*  9行目: `isnull`メソッドと`sum`メソッドを組み合わせて，各列の欠損値の数を計算して，`display`関数で表示する

In [34]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 43
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 35
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 33
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 25
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 26
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 20
display(pd.DataFrame(ship.isnull().sum()))

,0
乗客ID,0
生存,0
チケットクラス,0
性別,0
年齢,0
同乗1,0
同乗2,0
チケットID,0
運賃,0
部屋番号,687


### 性別列と生存列の関係
* ここで，性別（female/male）と生存者（生存列の値が1）の割合に関係があるかを調べる
* 具体的には，性別ごとの生存者の割合（生存列の算術平均と等価）を計算する
* ある列名の値によって分けられるグループごとに統計量等を計算する場合には，`groupby`メソッドを使う
* 詳細は「[ノートブック：SeriesとDataFrameの基本操作](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook_se_df.ipynb)」を参照
* 基本的な書式は`df.groupby('グループ分けしたい列インデックス').メソッド名()`となる．  
* 例えば，`ship`の中の性別列の値（female/male）ごとに算術平均を計算したい場合には，算術平均が計算不可能な列「チケットID」「部屋番号」「港コード」を削除したうえで，`ship.groupby('性別').mean()`と記述する
* ただし，この記述だとすべての列データに対して算術平均を計算することになるので，例えば，生存列の算術平均だけ取り出したい場合は，`ship.groupby('性別').mean()['生存']`と記述する
  
**［以下のコードの処理内容］**
*  1～2行目: ファイルの読み込み
*  3行目: `drop`メソッドを使って，`ship`から「チケットID」「部屋番号」「港コード」の列（`axis = 1`）を削除
*  4行目: `groupby`メソッドを使って，性別ごとの生存者の割合を計算し，`display`メソッドで表示
>*  生存者の割合は算術平均で計算できるので，`mean`メソッドを使う ⇒ `ship.groupby('性別').mean()`
>*  ここでは，生存列の算術平均だけ取り出したいので，`ship.groupby('性別').mean()['生存']`とする
>*  `groupby`メソッドによる集計結果はSeriesとなるので，それを`DataFrame`関数でDataFrameに変換してから表示

In [35]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
ship = ship.drop(['チケットID', '部屋番号', '港コード'], axis = 1)
display(pd.DataFrame(ship.groupby('性別').mean()['生存']))

,生存
性別,
female,0.742
male,0.189


### 性別列のダミー変数化（ワンホットエンコーディング）
* scikit-learnを使ったモデルの学習では，数値（boolを含む）のデータ列しか利用できないので，性別列のデータはこのままでは使えない
* そこで，性別列の「female」と「male」を1と0，あるいは`True`と`False`などの数値として扱える値に変換して，学習に利用できるようにする
* この操作のことを「**ダミー変数化**（またはワンホットエンコーディング）」と呼ぶ
* ダミー変数化には，pandasの`get_dummies`関数を用いる
>* 書式: `pd.get_dummies(ダミー変数化する列)`
>* 詳細は「[ノートブック：SeriesとDataFrameの基本操作](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook_se_df.ipynb)」を参照
* 具体的には，以下の処理を行う．
1.   female列とmale列を新たに追加
2.   female列: 性別列の値が「female」の場合は`True` または 1，「male」の場合は`False` または 0に変換
3.   male列: 性別列の値が「male」の場合は`True` または 1，「female」の場合は`False` または 0に変換
* 上記処理でfemale列とmale列の2列を追加しているが，一方の列が0（`False`）であれば，もう一方の列は必ず1（`True`）になる（逆も同様）
* したがって，モデルの学習に利用するのはどちらかの列で十分であることに注意しておく
  
**［以下のコードの処理内容］**
*  1～2行目: ファイルの読み込み
*  3行目: `get_dummies`関数を使って，性別列（`ship['性別']`）をダミー変数化し，その結果を変数`dummy`に代入
*  4行目: `display`関数を使って，変数`dummy`の内容を表示

In [36]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
dummy = pd.get_dummies(ship['性別'])
display(dummy)

,female,male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True
...,...,...
886,False,True
887,True,False
888,True,False
889,False,True


### DataFrameの結合
* 客船データの`ship`（DataFrame）と性別をダミー変数化した`dummy`（DataFrame）を結合して1つのDataFrameとして扱えるようにする
* DataFrameの結合には，`concat`関数を用いる
* 詳細は「[ノートブック：SeriesとDataFrameの基本操作](https://colab.research.google.com/github/yoshida-nu/lec_datascience/blob/main/doc/datascience_notebook_se_df.ipynb)」を参照
* `concat`関数の書式： `pd.concat([df1, df2], axis = [0 or 1])`
>*  `df1`と`df2`は結合するDataFrameの名前
>*  `axis = 0` ⇒ 行方向の結合（縦に並べて結合）
>*  `axis = 1` ⇒ 列方向の結合（横に並べて結合）
  
**［以下のコードの処理内容］**
*  1～2行目: ファイルの読み込み
*  3行目: `get_dummies`関数を使って，性別列（`ship['性別']`）をダミー変数化し，その結果を変数`dummy`に代入
*  4行目: `concat`関数を使って，2つのDataFrame`ship`と`dummy`を結合し，変数`ship`に代入
>*  列方向の結合（横に並べて結合）なので，`axis = 1` とする
*  5行目: `display`関数を使って，変数`ship`の内容を表示

In [37]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
dummy = pd.get_dummies(ship['性別'])
ship = pd.concat([ship, dummy], axis = 1)
display(ship)

,乗客ID,生存,チケットクラス,性別,年齢,同乗1,同乗2,チケットID,運賃,部屋番号,港コード,female,male
0,1,0,3,male,22.000,1,0,A/5 21171,7.250,NaN,S,False,True
1,2,1,1,female,38.000,1,0,PC 17599,71.283,C85,C,True,False
2,3,1,3,female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,True,False
3,4,1,1,female,35.000,1,0,113803,53.100,C123,S,True,False
4,5,0,3,male,35.000,0,0,373450,8.050,NaN,S,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.000,0,0,211536,13.000,NaN,S,False,True
887,888,1,1,female,19.000,0,0,112053,30.000,B42,S,True,False
888,889,0,3,female,NaN,1,2,W./C. 6607,23.450,NaN,S,True,False
889,890,1,1,male,26.000,0,0,111369,30.000,C148,C,False,True


### 説明変数に性別列（female列）を追加
* これまでと同様にして，客船データのDataFrame「`ship`」を訓練データ`(x_train, t_train)`とテストデータ`(x_test, t_test)`に分割する
* 先ほどは，説明変数に，チケットクラス列，年齢列，同乗1列，同乗2列，運賃列の5個を選択していたが，ここでは新たにfemale列を追加して合計6個の説明変数とする
* 年齢列の欠損値への対処も同様の処理を行う
  
**［以下のコードの処理内容］**
*  1～4行目: ファイルの読み込み，ダミー変数化とデータの結合
*  5～10行目: 年齢列の欠損値への対処
*  11行目: 説明変数の列名を要素とするリスト`['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female']`を変数`col`に代入
*  12行目: DataFrame `ship` から,`ship[col]`で，説明変数の列だけ取り出し変数`x`に代入
*  13行目: DataFrame `ship` から,`ship['生存']`で目的変数の列「生存」だけ取り出し変数`t`に代入
*  14行目: `model_selection`モジュールの`train_test_split`関数の読み込み
*  15行目: `train_test_split`関数を使って説明変数データ`x`と目的変数データ（正解データ）`t`を訓練データ（8割）とテストデータ（2割）にそれぞれ分割
*  16～23行目: 各データを表示

In [38]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
dummy = pd.get_dummies(ship['性別'])
ship = pd.concat([ship, dummy], axis = 1)
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 43
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 35
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 33
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 25
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 26
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 20
col = ['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female']
x = ship[col]
t = ship['生存']
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size = 0.2, random_state = random_seed)
print('================ x_train ================')
display(x_train)
print('================ t_train ================')
display(pd.DataFrame(t_train))
print('================ x_test ================')
display(x_test)
print('================ t_test ================')
display(pd.DataFrame(t_test))

================ x_train ================


,チケットクラス,年齢,同乗1,同乗2,運賃,female
329,1,16.000,0,1,57.979,True
749,3,31.000,0,0,7.750,False
203,3,45.500,0,0,7.225,False
421,3,21.000,0,0,7.733,False
97,1,23.000,0,1,63.358,False
...,...,...,...,...,...,...
98,2,34.000,0,1,23.000,True
322,2,30.000,0,0,12.350,True
382,3,32.000,0,0,7.925,False
365,3,30.000,0,0,7.250,False


================ t_train ================


,生存
329,1
749,0
203,0
421,0
97,1
...,...
98,1
322,1
382,0
365,0


================ x_test ================


,チケットクラス,年齢,同乗1,同乗2,運賃,female
172,3,1.000,1,1,11.133,True
524,3,26.000,0,0,7.229,False
452,1,30.000,0,0,27.750,False
170,1,61.000,0,0,33.500,False
620,3,27.000,1,0,14.454,False
...,...,...,...,...,...,...
388,3,26.000,0,0,7.729,False
338,3,45.000,0,0,8.050,False
827,2,1.000,0,2,37.004,False
773,3,26.000,0,0,7.225,False


================ t_test ================


,生存
172,1
524,0
452,0
170,0
620,0
...,...
388,0
338,1
827,1
773,0


### 分類木モデルの学習と評価
* 前のコードで作成したデータを使ってモデルの学習と評価を再度行う
  
**［以下のコードの処理内容］**
*  1～15行目: ファイルの読み込み，ダミー変数化とデータの結合，データの分割
*  16行目: `sklearn` (scikit-learn) の`tree`モジュールを読み込む 
*  17行目: `tree.DecisionTreeClassifier`で，分類木モデルの学習を行うためのオブジェクトを`tree`モジュールの`DecisionTreeClassifier`クラスから生成し，変数`model_tree_ship`に代入
>* `class_weight = 'balanced'`で不均衡データの影響を取り除く
>* `max_depth = 5`で最大深さを5とする
>* `random_state = random_seed`で乱数の種を指定する
*  18行目: `fit`メソッドで分類木モデルの学習を実行
>* 学習には訓練データ`x_train`, `t_train`を使う
*  19行目: `model_tree_ship.score(X = x_train, y = t_train)`で，訓練データから精度を計算して変数`score_train`に代入
*  20行目: `model_tree_ship.score(X = x_test, y = t_test)`で，テストデータに対する精度を計算して変数`score_test`に代入
*  21行目: `print`関数とf-stringを使って，訓練データに対する精度（`score_train`）とテストデータに対する精度（`score_test`）を表示
>* 「`:.3f`」で，小数点以下3桁まで表示

In [39]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
dummy = pd.get_dummies(ship['性別'])
ship = pd.concat([ship, dummy], axis = 1)
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 43
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 35
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 33
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 25
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 26
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 20
col = ['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female']
x = ship[col]
t = ship['生存']
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size = 0.2, random_state = random_seed)
from sklearn import tree
model_tree_ship = tree.DecisionTreeClassifier(class_weight = 'balanced', max_depth = 5, random_state = random_seed)
model_tree_ship.fit(x_train, t_train)
score_train = model_tree_ship.score(x_train, t_train)
score_test = model_tree_ship.score(x_test, t_test)
print(f'訓練データの精度={score_train:.3f} / テストデータの精度={score_test:.3f}')

訓練データの精度=0.862 / テストデータの精度=0.827


### 最大深さによる精度の変化の確認
* 先ほどと同様にして，訓練データとテストデータに対する各精度の推移を確認する
  
|**深さ**| **訓練データの精度** | **テストデータの精度** |
|:--|:--|:--|
|1|0.784|0.799|
|2|0.792|0.793|
|3|0.834|0.860|
|4|0.851|0.872|
|5|0.862|0.827|
|6|0.888|0.849|
|7|0.912|0.821|
|8|0.930|0.832|
|9|0.952|0.827|
|10|0.963|0.793|
|11|0.975|0.793|
|12|0.978|0.804|
|13|0.978|0.782|
|14|0.983|0.799|

<img src="./fig/accuracy_graph2.jpg" width="500">  


### 目的変数に影響している説明変数（特徴量）の確認
* 目的変数である生存列に影響を与えている説明変数を確認する
* 目的変数への影響の大きさを測る代表的な指標に特徴量重要度がある
* 特徴量重要度は0～1の値をとり，1に近いほどその列が目的変数の予測に与える影響が大きい
* 特徴量重要度は学習済みのオブジェクト`model_tree_ship`における`feature_importances_`属性を参照することで確認できる
  
**［以下のコードの処理内容］**
*  1～18行目: ファイルの読み込み，ダミー変数化とデータの結合，データの分割，モデルの学習
*  19行目: `display`関数で，`model_tree_ship`オブジェクトの`feature_importances_`属性を表示
>*  `DataFrame`関数で，`model_tree_ship.feature_importances_`（ndarray）をDataFrameに変換
>*  そのDataFrameの行名（`index`）を説明変数`x`の列名（`x.columns`），列名（`columns`）を「特徴量重要度」とした

In [40]:
url = 'https://bit.ly/43sIRq2'
ship = pd.read_csv(url)
dummy = pd.get_dummies(ship['性別'])
ship = pd.concat([ship, dummy], axis = 1)
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 43
ship.loc[(ship['チケットクラス'] == 1) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 35
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 33
ship.loc[(ship['チケットクラス'] == 2) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 25
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 0) & (ship['年齢'].isnull()), '年齢'] = 26
ship.loc[(ship['チケットクラス'] == 3) & (ship['生存'] == 1) & (ship['年齢'].isnull()), '年齢'] = 20
col = ['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female']
x = ship[col]
t = ship['生存']
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size = 0.2, random_state = random_seed)
from sklearn import tree
model_tree_ship = tree.DecisionTreeClassifier(class_weight = 'balanced', max_depth = 5, random_state = random_seed)
model_tree_ship.fit(x_train, t_train)
display(pd.DataFrame(model_tree_ship.feature_importances_, index = x.columns, columns = ['特徴量重要度']))

,特徴量重要度
チケットクラス,0.149
年齢,0.257
同乗1,0.056
同乗2,0.000
運賃,0.045
female,0.493


## 一連の分析
* 最後にまとめとして，これまでの分析を一括で行う
* 変数は一般的な名前に変更しているので注意する
  
**［分析手順］**
* ホールドアウト法による分類木モデルの学習を行う
>* 目的変数: 生存 
>* 説明変数（6つ）: チケットクラス, 年齢, 同乗1, 同乗2, 運賃, female
>* 訓練データ 80％，テストデータ 20％として分割
>* 不均衡データの影響を取り除く
>* 木の最大深さは 5 とする
>* 乱数の種は，すべて共通コードで定義している`random_state`を用いる
* 学習したモデルの評価を行う
>* 訓練データとテストデータに対する精度をそれぞれ計算する
>* 特徴量重要度を計算する
* 学習したモデルを使って予測を行う
>* 予測する客船データ(1): `['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female'] = [3, 27, 1, 0, 8, True]`
>* 予測する客船データ(2): `['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female'] = [1, 41, 0, 1, 30, False]`
  
**［以下のコードの処理内容］**
*  1行目: ファイルのURLを変数`url`に代入
*  2行目: pandasの`read_csv`関数を使って，ファイルをDataFrameとして読み込んで，変数`df`に代入
*  3行目: `get_dummies`関数を使って，性別列（`df['性別']`）をダミー変数化し，その結果を変数`dummy`に代入
*  4行目: `concat`関数を使って，2つのDataFrame`df`と`dummy`を結合し，変数`df`に代入
*  5行目: チケットクラスが1，かつ生存が0，かつ年齢が欠損している（`NaN`の）データの年齢列の値を43にする
>* 条件式は`(df['チケットクラス'] == 1) & (df['生存'] == 0) & (df['年齢'].isnull())`
*  6行目: チケットクラスが1，かつ生存が1，かつ年齢が欠損している（`NaN`の）データの年齢列の値を35にする
>* 条件式は`(df['チケットクラス'] == 1) & (df['生存'] == 1) & (df['年齢'].isnull())`
*  7行目: チケットクラスが2，かつ生存が0，かつ年齢が欠損している（`NaN`の）データの年齢列の値を33にする
>* 条件式は`(df['チケットクラス'] == 2) & (df['生存'] == 0) & (df['年齢'].isnull())`
*  8行目: チケットクラスが2，かつ生存が1，かつ年齢が欠損している（`NaN`の）データの年齢列の値を25にする
>* 条件式は`(df['チケットクラス'] == 2) & (df['生存'] == 1) & (df['年齢'].isnull())`
*  9行目: チケットクラスが3，かつ生存が0，かつ年齢が欠損している（`NaN`の）データの年齢列の値を26にする
>* 条件式は`(df['チケットクラス'] == 3) & (df['生存'] == 0) & (df['年齢'].isnull())`
*  10行目: チケットクラスが3，かつ生存が1，かつ年齢が欠損している（`NaN`の）データの年齢列の値を20にする
>* 条件式は`(df['チケットクラス'] == 3) & (df['生存'] == 1) & (df['年齢'].isnull())`
*  11行目: 説明変数の列名を要素とするリスト`['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female']`を変数`x_cols`に代入
*  12行目: 目的変数の列名`'生存'`を変数`t_col`に代入
*  13行目: DataFrame `df` から,`df[x_cols]`で，説明変数の列だけ取り出し変数`x`に代入
*  14行目: DataFrame `df` から,`df[t_col]`で目的変数の列だけ取り出し変数`t`に代入
*  15行目: `model_selection`モジュールの`train_test_split`関数の読み込み
*  16行目: `train_test_split`関数を使って説明変数データ`x`と目的変数データ（正解データ）`t`を訓練データ（80％）とテストデータ（20％）にそれぞれ分割
>* `random_state = random_seed`で乱数の種を指定する
*  17行目: `sklearn` (scikit-learn) の`tree`モジュールを読み込む 
*  18行目: `tree.DecisionTreeClassifier`で，分類木モデルの学習を行うためのオブジェクトを`tree`モジュールの`DecisionTreeClassifier`クラスから生成し，変数`model_tree`に代入
>* `class_weight = 'balanced'`で不均衡データの影響を取り除く
>* `max_depth = 5`で最大深さを5とする
>* `random_state = random_seed`で乱数の種を指定する
*  19行目: `fit`メソッドで分類木モデルの学習を実行
*  20行目: `model_tree.score(X = x_train, y = t_train)`で，訓練データから精度を計算して変数`score_train`に代入
*  21行目: `model_tree.score(X = x_test, y = t_test)`で，テストデータに対する精度を計算して変数`score_test`に代入
*  22行目: `print`関数とf-stringを使って，訓練データに対する精度（`score_train`）とテストデータに対する精度（`score_test`）を小数点以下3桁まで表示
>* 「`:.3f`」で，小数点以下3桁まで表示
*  23行目: `display`関数で，`model_tree`オブジェクトの`feature_importances_`属性（特徴量重要度）を表示
>*  `DataFrame`関数で，`model_tree.feature_importances_`（ndarray）をDataFrameに変換
>*  そのDataFrameの行名（`index`）を説明変数`x`の列名（`x.columns`），列名（`columns`）を「特徴量重要度」とした
*  24行目: 新しい2つの説明変数データ（`['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female']`）のリスト`[[3, 27, 1, 0, 8, True], [1, 41, 0, 1, 30, False]]`を変数`newdata`に代入
*  25行目: `predict`メソッドを用いて，特徴量`newdata`に対する予測を行い，その結果（`predict`メソッドの戻り値）を`print`関数で表示
>*  `f'予測結果： {model_tree.predict(X = newdata)}'` は f-string

In [41]:
url = 'https://bit.ly/43sIRq2'
df = pd.read_csv(url)
dummy = pd.get_dummies(df['性別'])
df = pd.concat([df, dummy], axis = 1)
df.loc[(df['チケットクラス'] == 1) & (df['生存'] == 0) & (df['年齢'].isnull()), '年齢'] = 43
df.loc[(df['チケットクラス'] == 1) & (df['生存'] == 1) & (df['年齢'].isnull()), '年齢'] = 35
df.loc[(df['チケットクラス'] == 2) & (df['生存'] == 0) & (df['年齢'].isnull()), '年齢'] = 33
df.loc[(df['チケットクラス'] == 2) & (df['生存'] == 1) & (df['年齢'].isnull()), '年齢'] = 25
df.loc[(df['チケットクラス'] == 3) & (df['生存'] == 0) & (df['年齢'].isnull()), '年齢'] = 26
df.loc[(df['チケットクラス'] == 3) & (df['生存'] == 1) & (df['年齢'].isnull()), '年齢'] = 20
x_cols = ['チケットクラス', '年齢', '同乗1', '同乗2', '運賃', 'female']
t_col = ['生存']
x = df[x_cols]
t = df[t_col]
from sklearn.model_selection import train_test_split
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size = 0.2, random_state = random_seed)
from sklearn import tree
model_tree = tree.DecisionTreeClassifier(class_weight = 'balanced', max_depth = 5, random_state = random_seed)
model_tree.fit(x_train, t_train)
score_train = model_tree.score(X = x_train, y = t_train)
score_test = model_tree.score(X = x_test, y = t_test)
print(f'訓練データの精度={score_train:.3f} / テストデータの精度={score_test:.3f}')
display(pd.DataFrame(model_tree.feature_importances_, index = x.columns, columns = ['特徴量重要度']))
newdata = [[3, 27, 1, 0, 8, True], [1, 41, 0, 1, 30, False]]
print(f'予測結果： {model_tree.predict(X = newdata)}')

訓練データの精度=0.862 / テストデータの精度=0.827


,特徴量重要度
チケットクラス,0.149
年齢,0.257
同乗1,0.056
同乗2,0.000
運賃,0.045
female,0.493


予測結果： [1 0]


# 分類木モデルの学習の実践 その2

## 用いるデータと分析の目的
* 例として，以下のURLにあるcsvファイルを用いる
>* https://bit.ly/41RgG2L
* このデータは，ある会社の従業員データベース一覧をイメージしている
>* 出典: 須出典:池田他: [実務で役立つPython機械学習入門 課題解決のためのデータ分析の基礎](https://www.shoeisha.co.jp/book/detail/9784798184890), 翔泳社, 2023
>* オリジナルのデータ: https://github.com/ml-pg-book/python-business-ml-starter
* 欠損値や外れ値はない
* 以降，このデータを「従業員データ」と呼ぶ


|**列名**| **意味** |
|:--|:--|
|leaving| 0: 離職してない / 1: 離職した |
|gender| 性別（女性 or 男性） |
|age| 年齢 |
|job_role| 職種 |
|job_satisfaction| 仕事に対する満足度（1～5の5段階評価） |
|environment_satisfaction| 職場環境に対する満足度（1～5の5段階評価） |
|over_time| 残業が多かったか（0: 少なかった / 1: 多かった） |
|income| 年収 |

* 従業員データを用いて，離職する（離職しそうな）従業員を予測することを考える
* そのために，分類木モデルを作成する
* これにより，どんなタイプの従業員が離職しやすいかがわかる

## データの確認
**［以下のコードの処理内容］**
*  1行目: ファイルのURLを変数`url`に代入
*  2行目: pandasの`read_csv`関数を使って，ファイルをDataFrameとして読み込んで，変数`df`に代入
*  3行目: `display`関数で`df`の内容を表示

In [42]:
url = 'https://bit.ly/41RgG2L'
df = pd.read_csv(url)
display(df)

,leaving,gender,age,job_role,job_satisfaction,environment_satisfaction,over_time,income,pred_leaving,monthly_income
0,1,女性,41,営業責任者,4,2,1,1198.600,0,1198.600
1,0,男性,49,研究員,2,3,0,1026.000,0,1026.000
2,1,男性,37,技術者,3,4,1,418.000,0,418.000
3,0,女性,33,研究員,3,4,1,581.800,0,581.800
4,0,男性,27,技術者,2,1,0,693.600,0,693.600
...,...,...,...,...,...,...,...,...,...,...
995,0,女性,43,研究員,3,1,1,816.200,0,816.200
996,0,女性,27,営業責任者,4,4,1,1153.800,0,1153.800
997,1,女性,27,研究員,3,4,1,478.800,0,478.800
998,0,男性,26,研究員,4,1,0,780.800,0,780.800


## 「leaving」列のデータの割合
* ここでは，列「leaving」のデータを目的変数（正解データ）として，分類木モデルの学習を行う
* まず「leving」列のデータ（0 or 1）がどのような割合になっているのかを確認する
  
**［以下のコードの処理内容］**
*  1～2行目: csvファイルをDataFrameとして読み込んで，変数`df`に代入
*  3行目: 列「leaving」の値の頻度を抽出し，その結果を`display`関数で表示
>* `df['leaving'].value_counts()`で，DataFrameである`df`の列「leaving」にどんな値がそれぞれ何個あるのかを調べる
>* `value_counts`メソッドの戻り値がSeriesなので，pandasの`DataFrame`関数でDataFrameに変換 ⇒ `pd.DataFrame(df['leaving'].value_counts())`
>* 変換したDataFrameを`display`関数で表示
  
*  実行結果から，従業員データが不均衡データであることが確認できる

In [43]:
url = 'https://bit.ly/41RgG2L'
df = pd.read_csv(url)
display(pd.DataFrame(df['leaving'].value_counts()))

,count
leaving,
0,833
1,167


## 一連の分析
* 以下の手順にしたがって分析を行う
  
**［分析手順］**
* ホールドアウト法による分類木モデルの学習を行う
>* 目的変数: leaving 
>* 説明変数（5つ）: age, job_satisfaction, environment_satisfaction, over_time, income
>* 訓練データ 70％，テストデータ 30％として分割
>* 不均衡データの影響を取り除く
>* 木の最大深さは 4 とする
>* 乱数の種は，すべて共通コードで定義している`random_state`を用いる
* 学習したモデルの評価を行う
>* 訓練データとテストデータに対する精度をそれぞれ計算する
>* 特徴量重要度を計算する
* 学習したモデルを使って予測を行う
>* 予測する従業員データ(1): `['age', 'job_satisfaction', 'environment_satisfaction', 'over_time', 'income'] = [30, 3, 1, 0, 726]`
>* 予測する従業員データ(2): `['age', 'job_satisfaction', 'environment_satisfaction', 'over_time', 'income'] = [50, 3, 3, 0, 986]`
  
**［以下のコードの処理内容］**
*  1行目: ファイルのURLを変数`url`に代入
*  2行目: pandasの`read_csv`関数を使って，ファイルをDataFrameとして読み込んで，変数`df`に代入
*  3行目: 説明変数の列名を要素とするリスト`['age', 'job_satisfaction', 'environment_satisfaction', 'over_time', 'income']`を変数`x_cols`に代入
*  4行目: 目的変数の列名`'leaving'`を変数`t_col`に代入
*  5行目: DataFrame `df` から,`df[x_cols]`で，説明変数の列だけ取り出し変数`x`に代入
*  6行目: DataFrame `df` から,`df[t_col]`で目的変数の列だけ取り出し変数`t`に代入
*  7行目: `model_selection`モジュールの`train_test_split`関数の読み込み
*  8行目: `train_test_split`関数を使って説明変数データ`x`と目的変数データ（正解データ）`t`を訓練データ（70％）とテストデータ（30％）にそれぞれ分割
*  9行目: `sklearn` (scikit-learn) の`tree`モジュールを読み込む 
*  10行目: `tree.DecisionTreeClassifier`で，分類木モデルの学習を行うためのオブジェクトを`tree`モジュールの`DecisionTreeClassifier`クラスから生成し，変数`model_tree`に代入
*  11行目: `fit`メソッドで分類木モデルの学習を実行
*  12行目: `model_tree.score(X = x_train, y = t_train)`で，訓練データに対する精度を計算して変数`score_train`に代入
*  13行目: `model_tree.score(X = x_test, y = t_test)`で，テストデータに対する精度を計算して変数`score_test`に代入
*  14行目: `print`関数とf-stringを使って，訓練データに対する精度（`score_train`）とテストデータに対する精度（`score_test`）を小数点以下3桁まで表示
*  15行目: `display`関数で，`model_tree`オブジェクトの`feature_importances_`属性（特徴量重要度）を表示
*  16行目: 新しい2つの説明変数データ（`['age', 'job_satisfaction', 'environment_satisfaction', 'over_time', 'income']`）のリスト`[[30, 3, 1, 0, 726], [50, 3, 3, 0, 986]]`を変数`newdata`に代入
*  17行目: `predict`メソッドを用いて，特徴量`newdata`に対する予測を行い，その結果（`predict`メソッドの戻り値）を`print`関数で表示
  
**［問題］**
* 以下のコードの 「**# ここにコードを記述**」 に適切なキーワードを入れて，上記処理を行うコードを完成せよ

In [ ]:
url = 'https://bit.ly/41RgG2L'
df = pd.read_csv(url)
# ここにコードを記述
# ここにコードを記述
x = df[x_cols]
t = df[t_col]
from sklearn.model_selection import train_test_split
# ここにコードを記述
from sklearn import tree
# ここにコードを記述
model_tree.fit(X = x_train, y = t_train)
score_train = model_tree.score(X = x_train, y = t_train)
score_test = model_tree.score(X = x_test, y = t_test)
print(f'訓練データの精度={score_train:.3f} / テストデータの精度={score_test:.3f}')
display(pd.DataFrame(model_tree.feature_importances_, index = x.columns, columns = ['特徴量重要度']))
# ここにコードを記述
print(f'予測結果： {model_tree.predict(X = newdata)}')